# DataSet APIの動作確認

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
sess = tf.InteractiveSession()

In [3]:
def read_csv(filename):
    """
    データファイルの定義ファイルを読み込むgeneratorを作る
    """
    with open(filename, 'r') as f:
        for line in f.readlines():
            record = line.rstrip().split(',')
            image_file = record[0]
            label = int(record[1])
            yield image_file, label

def read_image(image_file, label):
    contents = tf.read_file(image_file)
    image = tf.image.decode_image(contents)  # 画像データを[0,1)の範囲に変換
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

In [4]:
file_path = 'image_list_test.csv'
n_batch = 5

In [ ]:
#image_list_file = tf.placeholder(tf.string, shape=[])
#def generator(f_path): return read_csv(f_path)  # generator

def generator(): return read_csv(file_path)  # generator

dataset = tf.data.Dataset.from_generator(
    #lambda:generator(image_list_file),
    generator, 
    output_types=(tf.string, tf.int64),
    output_shapes=(tf.TensorShape([]), tf.TensorShape([]))
    ).map(read_image)
dataset = dataset.shuffle(10000)
dataset = dataset.batch(n_batch)
iterator = dataset.make_initializable_iterator()
#images, labels = iterator.get_next()
values = iterator.get_next()

In [ ]:
#init = tf.global_variables_initializer()
#sess.run(init)
#sess.run(iterator.initializer, feed_dict={image_list_file:file_path})
sess.run(iterator.initializer)

In [ ]:
try:
    while True:
        xs, ys = sess.run(values)
        print('{}, {}'.format(xs.shape, ys))
except tf.errors.OutOfRangeError:
    pass

In [5]:
ff = tf.placeholder(tf.string)

class TestDataSet:
    def input_fn(self):
        def generator(file_path): return read_csv(file_path)  # generator
        
        dataset = tf.data.Dataset.from_generator(
            lambda:generator(ff),
            #generator, 
            output_types=(tf.string, tf.int64),
            output_shapes=(tf.TensorShape([]), tf.TensorShape([]))
        ).map(read_image)
        dataset = dataset.shuffle(10000)
        dataset = dataset.batch(n_batch)
        iterator = dataset.make_initializable_iterator()
        self.iterator = iterator
        images, labels = iterator.get_next()
        return images, labels

In [6]:

ds = TestDataSet()
imgs, labs = ds.input_fn()

In [7]:
sess.run(ds.iterator.initializer, feed_dict={ff:file_path})
try:
    while True:
        xs = sess.run(imgs)
        print('{}'.format(xs.shape))
except tf.errors.OutOfRangeError:
    pass

InvalidArgumentError: TypeError: expected str, bytes or os.PathLike object, not Tensor
Traceback (most recent call last):

  File "/Users/tokita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 157, in __call__
    ret = func(*args)

  File "/Users/tokita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 382, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-3-202308b73d49>", line 5, in read_csv
    with open(filename, 'r') as f:

TypeError: expected str, bytes or os.PathLike object, not Tensor


	 [[Node: PyFunc = PyFunc[Tin=[DT_INT64], Tout=[DT_STRING, DT_INT64], token="pyfunc_1"](arg0)]]
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[<unknown>, [?]], output_types=[DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]